In [1]:
!pip install datasets
!pip install convokit
!pip install psutil

Defaulting to user installation because normal site-packages is not writeable
     -------------------------------------- 451.7/451.7 kB 5.6 MB/s eta 0:00:00
     --------------------------------------- 20.3/20.3 MB 23.4 MB/s eta 0:00:00
     ---------------------------------------- 132.9/132.9 kB ? eta 0:00:00
     ------------------------------------- 323.5/323.5 kB 19.6 MB/s eta 0:00:00
     ---------------------------------------- 56.7/56.7 kB ? eta 0:00:00
  Using cached dill-0.3.6-py3-none-any.whl (110 kB)


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
from convokit import Corpus, download

subreddit = "subreddit-wallstreetbets"
corpus = Corpus(download(subreddit))

Dataset already exists at C:\Users\admin\.convokit\downloads\subreddit-wallstreetbets


In [3]:
from datasets import Dataset

def map_utterance(utterance):
    return {
        'utterance' : utterance.text,
        'score': utterance.meta['score']
    }

corpus_utterances = [utterance for utterance in corpus.utterances.values()]
utterances = list(map(lambda utterance : utterance.text, corpus_utterances))
scores = list(map(lambda utterance : utterance.meta['score'], corpus_utterances))


dataset = Dataset.from_dict({"utterance": utterances, "scores": scores})
dataset

Dataset({
    features: ['utterance', 'scores'],
    num_rows: 2890401
})

In [19]:
# Data processing

filters = ["", "[removed]", "[deleted]"]
max_length = 500
min_length = 16

def reddit_comment_filter(utterance):
    return utterance['utterance'] not in filters and len(utterance['utterance']) < max_length

# TODO: You can add other things like score filters here

def reddit_score_filter(utterance):
    return utterance['scores'] > 5

def reddit_min_length_filter(utterance):
    return len(utterance['utterance']) > min_length

shuffled_dataset = dataset.shuffle(seed=10617)
filtered_dataset = shuffled_dataset.filter(reddit_comment_filter).filter(reddit_min_length_filter)
train_test_split = filtered_dataset.train_test_split(test_size=0.1)
train_test_split

  0%|          | 0/2891 [00:00<?, ?ba/s]

  0%|          | 0/2434 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['utterance', 'scores'],
        num_rows: 1920926
    })
    test: Dataset({
        features: ['utterance', 'scores'],
        num_rows: 213437
    })
})

In [20]:
# Sort
num_train = train_test_split["train"].shape[0]
train_test_split["train"] = train_test_split["train"].sort("scores", reverse=True)[:int(0.25*num_train)]
train_dataset = Dataset.from_dict(train_test_split["train"]).shuffle(seed=10617)
train_test_split["train"] = train_dataset
train_test_split

DatasetDict({
    train: Dataset({
        features: ['utterance', 'scores'],
        num_rows: 480231
    })
    test: Dataset({
        features: ['utterance', 'scores'],
        num_rows: 213437
    })
})

In [ ]:
# This should be top upvoted comment now
#train_test_split["train"][0]

In [21]:
# Save our test/train split

# experiment_name = "sorted_by_upvotes"
# experiment_name = "sorted_by_upvotes_positive"
experiment_name = "top_25"

train_test_split.save_to_disk(f"./train_test_{experiment_name}_{subreddit}")

Flattening the indices:   0%|          | 0/481 [00:00<?, ?ba/s]

Flattening the indices:   0%|          | 0/214 [00:00<?, ?ba/s]